In [1]:
import pandas as pd 
import os 
os.chdir('C:/Users/Usuario/Desktop/ASTYANAX_EYE/Superficie/2_ProteinOrtho/2_ProteinOrtho_IDs')

Al archivo obtenido de ProteinOrtho (Ast-Dmel) se agregaron los ID's (a2, b2) del proteoma obtenido a traves del NCBI

In [2]:
ortho = pd.read_csv('Ortho_Ast_Dmel.csv') 
ortho.head()

,Dmel,a,a2,Ast,b,b2,evalue_ab,bitscore_ab,evalue_ba,bitscore_ba
0,Drosophila_melanogaster,NP_001014460.1,SAU,Astyanax_mexicanus,XP_007250532.1,GOLPH3A,9.400000e-111,399.0,4.580000e-111,399.0
1,Drosophila_melanogaster,NP_001014460.1,SAU,Astyanax_mexicanus,XP_007228899.2,GOLPH3B,3.850000e-107,387.0,1.890000e-107,387.0
2,Drosophila_melanogaster,NP_001014461.1,CG9967,Astyanax_mexicanus,XP_007229687.1,MKRN2OS.2,3.380000e-20,98.2,1.270000e-20,98.2
3,Drosophila_melanogaster,NP_001014462.1,RBP9,Astyanax_mexicanus,XP_022526743.1,ELAVL4,1.120000e-109,396.0,5.530000e-110,396.0
4,Drosophila_melanogaster,NP_001014462.1,RBP9,Astyanax_mexicanus,XP_022524325.1,ELAVL2,4.490000e-109,394.0,1.960000e-109,394.0


In [3]:
ortho.shape

(45115, 10)

In [4]:
nod= pd.read_csv('Dmel_nodos.csv')

In [5]:
nod.head()

,Model_nodos,NCBI_Product_accesion
0,14-3-3EPSILON,NP_732309.1
1,14-3-3ZETA,NP_476885.2
2,18W,NP_476814.1
3,2MIT,NP_001262529.1
4,5-HT1A,NP_001356890.1


In [5]:
nod.shape

(4353, 2)

In [6]:
# Asegúrate de que no hay espacios extra o diferencias en los valores
nod['NCBI_Product_accesion'] = nod['NCBI_Product_accesion'].astype(str).str.strip()
ortho['a'] = ortho['a'].astype(str).str.strip()

# Filtrado: conservar solo las filas de 'ortho' que tengan coincidencias en 'nod'
filtrado = ortho[ortho['a'].isin(nod['NCBI_Product_accesion'])]

# Mostrar los primeros resultados
filtrado.head()


,Dmel,a,a2,Ast,b,b2,evalue_ab,bitscore_ab,evalue_ba,bitscore_ba
101,Drosophila_melanogaster,NP_001014522.2,MP20,Astyanax_mexicanus,XP_049332440.1,TAGLN2,4.580000e-29,127.0,2.380000e-29,127.0
109,Drosophila_melanogaster,NP_001014529.1,U3-55K,Astyanax_mexicanus,XP_007254609.3,RRP9,2.110000e-71,269.0,1.050000e-71,269.0
110,Drosophila_melanogaster,NP_001014529.1,U3-55K,Astyanax_mexicanus,XP_007254607.3,RRP9,1.350000e-69,263.0,6.750000e-70,263.0
129,Drosophila_melanogaster,NP_001014549.1,SLIK,Astyanax_mexicanus,XP_007252332.3,SLKA,5.240000e-114,412.0,2.290000e-114,412.0
130,Drosophila_melanogaster,NP_001014552.1,ZIP,Astyanax_mexicanus,XP_022530909.1,MYH9A,0.000000e+00,1260.0,0.000000e+00,1260.0


In [7]:
filtrado.shape

(5655, 10)

In [10]:
#Unión de nodos del modelo con ortólogos
merged = pd.merge(nod, filtrado, left_on='NCBI_Product_accesion', right_on='a', how='left')

In [11]:
merged.head()

,Model_nodos,NCBI_Product_accesion,Dmel,a,a2,Ast,b,b2,evalue_ab,bitscore_ab,evalue_ba,bitscore_ba
0,14-3-3EPSILON,NP_732309.1,Drosophila_melanogaster,NP_732309.1,14-3-3EPSILON,Astyanax_mexicanus,XP_007245994.1,YWHAE1,8.180000e-114,409.0,3.930000e-114,409.0
1,14-3-3EPSILON,NP_732309.1,Drosophila_melanogaster,NP_732309.1,14-3-3EPSILON,Astyanax_mexicanus,XP_007229712.1,YWHAE2,6.540000e-113,406.0,3.140000e-113,406.0
2,14-3-3EPSILON,NP_732309.1,Drosophila_melanogaster,NP_732309.1,14-3-3EPSILON,Astyanax_mexicanus,XP_007245995.1,YWHAE1,1.050000e-111,402.0,4.740000e-112,402.0
3,14-3-3EPSILON,NP_732309.1,Drosophila_melanogaster,NP_732309.1,14-3-3EPSILON,Astyanax_mexicanus,XP_049323473.1,YWHAE2,4.190000e-111,400.0,1.890000e-111,400.0
4,14-3-3ZETA,NP_476885.2,Drosophila_melanogaster,NP_476885.2,14-3-3ZETA,Astyanax_mexicanus,XP_007241340.1,LOC103023259,4.160000e-105,380.0,2.020000e-105,380.0


In [17]:
# 4. Agrupar por nodo y accession ID — mantener lista de tuplas (ID, símbolo)
grouped = merged.groupby(['Model_nodos', 'a']).apply(
    lambda df: list(zip(df['b'], df['b2']))
).reset_index(name='Astyanax_ortologos')

# 5. Añadir símbolo del gen de los IDs de referencia
symbol_map = filtrado[['a', 'a2']].drop_duplicates().set_index('a')['a2']
grouped['Symbol'] = grouped['a'].map(symbol_map)

# 6. Renombrar columna 'a' a 'NCBI_Product_accesion' para mantener consistencia
grouped = grouped.rename(columns={'a': 'NCBI_Product_accesion'})

# 7. Reordenar columnas
grouped = grouped[['Model_nodos', 'NCBI_Product_accesion', 'Astyanax_ortologos']]


C:\Users\Usuario\AppData\Local\Temp\ipykernel_16848\538579209.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = merged.groupby(['Model_nodos', 'a']).apply(


In [18]:
grouped.head()

,Model_nodos,NCBI_Product_accesion,Astyanax_ortologos
0,14-3-3EPSILON,NP_732309.1,"[(XP_007245994.1, YWHAE1), (XP_007229712.1, YW..."
1,14-3-3ZETA,NP_476885.2,"[(XP_007241340.1, LOC103023259), (XP_049335888..."
2,5-HT7,NP_524599.1,"[(XP_007256792.3, HTR1AB), (XP_007234230.1, HT..."
3,ABI,NP_001247090.1,"[(XP_022533518.1, ABI1B), (XP_022533520.1, ABI..."
4,AC3,NP_610116.2,"[(XP_022541047.2, LOC103034748), (XP_022541050..."


In [20]:
grouped.to_csv('Dmel_Dic_Orto.csv')

In [21]:
merged.to_csv('Dmel_Dic_merged.csv')